In [1]:
import pandas as pd
from IPython.display import display

In [ ]:
df = pd.read_csv(
    '../data/test_head.csv',
    parse_dates=[1,2,3],
    date_parser=lambda d: datetime.strptime(d, '%m/%d/%Y %H:%M:%S %p')
)

In [ ]:
with pd.option_context("display.max_columns", 50):
    display(df.head(2))

## Mongo time

In [ ]:
from pymongo import MongoClient

In [ ]:
mongo_client = MongoClient(connect=False)
coll = mongo_client['311']['done']
df2 = pd.DataFrame(list(coll.find()))
mongo_client.close()

In [ ]:
df2 = df2.drop('_id', axis=1)
df2['description'] = df2.description.apply(lambda txt: pd.np.NaN if txt == '' else txt)

In [ ]:
df2.head(2)

In [ ]:
from_aws = set(df2.case_enquiry_id)
from_csv = set(df.CASE_ENQUIRY_ID[df.TYPE.isin(('Missed Trash/Recycling/Yard Waste/Bulk Item','Street Light Outages','Request for Pothole Repair','Sidewalk Repair (Make Safe','Graffiti Removal','Tree Maintenance Requests','Sign Repair','Pick up Dead Animal','Rodent Activity','Abandoned Vehicles','Traffic Signal Repair','CE Collection','Sidewalk Repair','Traffic Signal Inspection','Tree Emergencies','General Lighting Request','Highway Maintenance','Equipment Repair','New Sign  Crosswalk or Pavement Marking','PWD Graffiti','Ground Maintenance','Missing Sign','Street Light Knock Downs','Major System Failure','Unshoveled Sidewalk','Bed Bugs','Space Savers','Mice Infestation - Residential','Needle Pickup','Request for Snow Plowing (Emergency Responder','Sidewalk Cover / Manhole','Roadway Repair','Catchbasin','Pavement Marking Maintenance'))])

In [ ]:
len(from_csv), len(from_aws)

In [ ]:
from_aws.difference(from_csv)

In [ ]:
from_csv.difference(from_aws)

yay!

## Merging the two DFs

In [ ]:
new_df = df.merge(df2, how='left', left_on='CASE_ENQUIRY_ID', right_on='case_enquiry_id')
new_df.head(2)

In [ ]:
new_df.shape, df.shape, df2.shape

In [ ]:
new_df.CLOSED_DT - new_df.OPEN_DT

In [ ]:
new_df['completion_time'] = new_df.CLOSED_DT - new_df.OPEN_DT

In [ ]:
new_df.to_pickle('../data/data_w_descs.pkl')